In [9]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://ice.enap.gov.br/files/Posicoes.xlsx"
POSITION_PATH = os.path.join("datasets", "positions")
POSITION_URL = DOWNLOAD_ROOT + "datasets/iris/position.tgz"

def fetch_position_data(position_url=POSITION_URL, position_path=POSITION_PATH):
    if not os.path.isdir(position_path):
        os.makedirs(position_path)
    tgz_path = os.path.join(position_path, "position.tgz")
    urllib.request.urlretrieve(position_url, tgz_path)
    position_tgz = tarfile.open(tgz_path)
    position_tgz.extractall(path=position_path)
    position_tgz.close()

In [10]:
#fetch_position_data()

import pandas as pd
import numpy as np

def load_position_data(position_path=POSITION_PATH):
    xlsx_path = os.path.join(position_path, "position.xlsx")
    df =   pd.read_excel(xlsx_path, engine = 'openpyxl')
    return pd.read_excel(xlsx_path, engine = 'openpyxl',header = 4 , usecols = "A,B,T,AG,AS,AX,BM,CA,CK,CO",
                         names=['UF', 'Cidade',df.iloc[0][19],df.iloc[0][32],df.iloc[0][44],df.iloc[0][49],df.iloc[0][64],df.iloc[0][78],df.iloc[0][88],df.iloc[2][92]])

In [11]:
cities = load_position_data()
cities.head()

,UF,Cidade,Índice de Ambiente Regulatório,Índice de Infraestrutura,Índice de Mercado,Índice de Acesso a Capital,Índice de Inovação,Índice de Capital Humano,Índice de Cultura,ICE 2020
0,PA,Ananindeua,65,98,55,88,82,88,22,88
1,GO,Anápolis,75,42,67,62,64,71,31,75
2,GO,Aparecida de Goiânia,51,56,84,94,54,93,26,86
3,SE,Aracaju,11,70,81,15,68,44,21,66
4,SP,Bauru,29,39,47,30,35,18,48,35


In [12]:
from geopy.geocoders import Nominatim
nom = Nominatim(scheme="http", user_agent="ICE_2020")

In [13]:
cities['Coordenadas']=(cities['Cidade']+","+cities['UF']+", Brazil").apply(nom.geocode)

In [14]:
cities['Latitude']=cities['Coordenadas'].apply(lambda x: x.latitude)
cities['Longitude']=cities['Coordenadas'].apply(lambda x: x.longitude)

In [15]:
from shapely.geometry import Point
cities['Geometria'] = None

for index,row in cities.iterrows():
    cities.loc[index, 'Geometria'] = Point(row.Longitude, row.Latitude)

cities.drop(['Coordenadas'], inplace=True, axis=1)

In [16]:
#import geopandas as gpd
#gdf_cities = gpd.GeoDataFrame(cities, geometry='Geometria')

In [17]:
#BRASIL_PATH = os.path.join("datasets", "positions")
#def load_brasil_data(brasil_path=BRASIL_PATH):
#    shp_path = os.path.join(brasil_path, "UFEBRASIL.shp")
#    return gpd.read_file(shp_path)

In [18]:
import folium
media_latitude = cities['Latitude'].mean()
media_longitude = cities['Longitude'].mean()

mapa = folium.Map(locations=[media_latitude, media_longitude], tiles= 'cartodbpositron')

def circle_maker(x):
    folium.Circle(location=[x[10],x[11]],
                 radius=8000,fill=True,
                 popup='Cidade: {} \n Posição no Rank ICE: {}'.format(x[1],x[9])).add_to(mapa)

In [19]:
cities.apply(lambda x: circle_maker(x), axis=1)
mapa.save('mapas/indice_ice_2020.html')